In [1]:
import math
import numpy as np
import numba
import matplotlib.pyplot as plt
from numba import jit
import time
import scipy.integrate as integrate
import scipy.constants as cnt
from scipy.optimize import minimize
from decimal import Decimal
import multiprocessing as mp

#### Sequences for proteins full H1 and full prota

In [2]:
h1 = {
    "seq": "TENSTSAPAAKPKRAKASKKSTDHPKYSDMIVAAIQAEKNRAGSSRQSIQKYIKSHYKVGENADSQIKLSIKRLVTTGVLKQTKGVGASGSFRLAKSDEPKKSVAFKKTKKEIKKVATPKKASKPKKAASKAPTKKPKATPVKKAKKKLAATPKKAKKPKTVKAKPVKASKPKKAKPVKPKAKSSAKRAGKKK",
#     "seq": "MTENSTSAPAAKPKRAKASKKSTDHPKYSDMIVAAIQAEKNRAGSSRQSIQKYIKSHYKVGENADSQIKLSIKRLVTTGVLKQTKGVGASGSFRLAKSDEPKKSVAFKKTKKELKKVATPKKASKPKKAASKAPTKKPKATPVKKTKKELKKVATPKKAKKPKTVKAKPVKASKPKKAKPVKPKAKSSAKRAGKKKHHHHHH",
#     "seq": "CTENSTSAPAAKPKRAKASKKSTDHPKYSDMIVAAIQAEKNRAGSSRQSIQKYIKSHYKVGENADSQIKLSIKRLVTTGVLKQTKGVGASGSFRLAKSDEPKKSVAFKKTKKEIKKVATPKKASKPKKAASKAPTKKPKATPVKKAKKKLAATPKKAKKPKTVKAKPVKASKPKKAKPVKPKAKSSAKRAGKKKGGPR",
    "qseq": [],
    "q": +53
}
protalpha = {
#     "seq": "GPSDAAVDTSSEITTKDLKEKKEVVEEAENGRDAPANGNAENEENGEQEADNEVDEEEEEGGEEEEEEEEGDGEEEDGDEDEEAESATGKRAAEDDEDDDVDTKKQKTDEDD",
#     "seq": "GSYMSDAAVDTSSEITTKDLKEKKEVVEEAENGRDAPANGNAENEENGEQEADNEVDEEEEEGGEEEEEEEEGDGEEEDGDEDEEAESATGKRAAEDDEDDDVDTKKQKTDEDD",
#     "seq": "GPSDAAVDTSSEITTKDLKEKKEVVEEAENGRDAPANGNAENEENGEQEADNEVDEEEEEGGEEEEEEEEGDGEEEDGDEDEEAESATGKRAAEDDEDDDVDTKKQKTDEDD",
    "seq": "SDAAVDTSSEITTKDLKEKKEVVEEAENGRDAPANGNAENEENGEQEADNEVDEEEEEGGEEEEEEEEGDGEEEDGDEDEEAESATGKRAAEDDEDDDVDTKKQKTDEDD",
    "qseq": [],
    "q": -44
}

#### Explicit integrand


In [3]:
@jit(nopython=True)
def integrand(x, a, b, kuhn, kappa, lb):
    v = (x / (x ** 2.0 + kappa ** 2)) ** 2.0
    s = 0
    for qa1 in range(a.shape[0]):
        for qa2 in range(a.shape[0]):
            for qb1 in range(b.shape[0]):
                for qb2 in range(b.shape[0]):
                    s += a[qa1][0] * a[qa2][0] * b[qb1][0] * b[qb2][0] * math.exp(
                        -(1.0 / 6.0) * (abs(a[qa1][1] - a[qa2][1]) + abs(b[qb1][1] - b[qb2][1])) * (x * kuhn) ** 2.0)
#     return - 4.0 * v * s * (lb ** 2.0) / kappa ** 4 * 10**(-27)
    return v * s

#### Explicitly find Kd

In [4]:
def findKd(protA, protB, lb, kappa, kuhn, order = 1, ranget=np.inf):
    B2_O1 = 4.0 * math.pi * lb * protA["q"] * protB["q"] / (kappa ** 2.0) 
    B2_O2 = 0
    integral = [0]
    if type(kuhn) is np.ndarray:
        kuhn = kuhn[0]
    if order==2:
        integral = integrate.quad(integrand, 
                                  a=0, 
                                  b=ranget, 
                                  args=(protA["qseq"], protB["qseq"], kuhn, kappa, lb),
                                  epsabs=0,
                                  epsrel=1.49e-10)
#         integral = [740]
        B2_O2 = - 4.0 * (integral[0]) * (lb ** 2.0) 
#         B2_O2 = integral[0]
        
    B2 = (B2_O1 + B2_O2)*10**(-27)          # In m**3
    kD = -(1/(B2*cnt.Avogadro))*10**(-3)    # In L 
    return kD,  integral[0], B2_O1, B2_O2

#### Kd Opt function

In [5]:
def KdOpt(x, KdExp, protA, protB, lb, kappa, order = 1, ranget=1e8):
    Kd = findKd(protA, protB, lb, kappa, x, order = 2, ranget=1e8)
    return (Kd[0] - KdExp)**2

#### Main Run

In [12]:
expkD0 = np.array([142, 189, 223, 257, 300]) * 10 ** -6
expkD1 = np.array([3.41, 5.09, 6.46, 7.94, 9.95]) * 10 ** -6

c     = np.array([165, 220, 260, 300, 350]) * 10 ** -3
I     = c[4] * 10**(-27) * 10 ** 3                                                                    # Salt Concentration in Molar
lb    = cnt.e * cnt.e / (4 * math.pi * cnt.epsilon_0 * 80 * cnt.Boltzmann * 300)*10**9  # Bjerrum Length (at T=300K)
kappa = np.sqrt(8 * math.pi * lb * I * cnt.Avogadro)                          # Debye Wavenumber
kuhn  = 0.3768                                                                   # Kuhn Length

h1["qseq"]        = []
protalpha["qseq"] = []

charge_dict = {'D': -1, 'E': -1, 'R': +1, 'K': +1}  # This dict reproduces data from the paper
    
for prot in [h1, protalpha]:
    for i, d in enumerate(prot["seq"]):
        if charge_dict.get(d, None) is not None:
            prot["qseq"].append([charge_dict[d], i + 1])
    prot["qseq"] = np.array(prot["qseq"])
print("="*20,"CHARGE CHECK","="*20)
print("= H1 : Expected ",h1["q"],"From QSEQ",np.sum(h1["qseq"][:,0], axis=0))
print("= Protalpha : Expected", protalpha["q"],"From QSEQ",np.sum(protalpha["qseq"][:,0], axis=0))
print("="*20,"CHARGE CHECK","="*20,"\n")
kappa = np.sqrt(8 * math.pi * lb * I * cnt.Avogadro)
Kd, integ, B2_O1, B2_O2 = findKd(h1, protalpha, lb, kappa, kuhn, order=2, ranget=np.inf)

print("INTEGRAL RESULT", integ)
print("B2_O1 ",B2_O1)
print("B2_O2 ",B2_O2)
print("Kd (um) ", Kd * 10**6)

==================== CHARGE CHECK ====================
= H1 : Expected  53 From QSEQ 53
= Protalpha : Expected -44 From QSEQ -44
==================== CHARGE CHECK ==================== 

INTEGRAL RESULT 83572.03424139663
B2_O1  -5531.967203251638
B2_O2  -162052.81906192406
Kd (um)  9.908650286426543


#### Plot integrand

In [36]:
c     = np.array([165, 220, 260, 300, 350]) * 10 ** -3
I     = c[0] *10**(-27)                                                                     # Salt Concentration in Molar
lb    = cnt.e * cnt.e / (4 * math.pi * cnt.epsilon_0 * 80 * cnt.Boltzmann * 300)*10**9  # Bjerrum Length (at T=300K)
kappa = np.sqrt(8 * math.pi * lb * I * cnt.Avogadro)                          # Debye Wavenumber
kuhn  = 0                                                                    # Kuhn Length


h1["qseq"]        = []
protalpha["qseq"] = []


charge_dict = {'D': -1, 'E': -1, 'R': +1, 'K': +1}  # This dict reproduces data from the paper
    
for prot in [h1, protalpha]:
    for i, d in enumerate(prot["seq"]):
        if charge_dict.get(d, None) is not None:
            prot["qseq"].append([charge_dict[d], i + 1])
    prot["qseq"] = np.array(prot["qseq"])

a = h1["qseq"]
b = protalpha["qseq"]
print(a.shape[0]**2*b.shape[0]**2)                    
s=0
for qa1 in range(a.shape[0]):
    for qa2 in range(a.shape[0]):
        for qb1 in range(b.shape[0]):
            for qb2 in range(b.shape[0]):
                s += a[qa1,0] * a[qa2,0] * b[qb1,0] * b[qb2,0]

print(kappa, lb, s)

20647936
0.04169857973374781 0.6962541756194208 5438224


#### Kuhn length optimization

In [ ]:
expkD0 = np.array([142, 189, 223, 257, 300]) * 10 ** -6
expkD1 = np.array([3.41, 5.09, 6.46, 7.94, 9.95]) * 10 ** -6

c     = np.array([165, 220, 260, 300, 350]) * 10 ** -3
I     = c[0] *10**(-27)                                                                     # Salt Concentration in Molar
lb    = cnt.e * cnt.e / (4 * math.pi * cnt.epsilon_0 * 80 * cnt.Boltzmann * 300)*10**9      # Bjerrum Length (at T=300K)
kappa = np.sqrt(4 * math.pi * lb * (2 * I * cnt.Avogadro))                                  # Debye Wavenumber
kuhn  = 1                                                                                # Kuhn Length

h1["qseq"]        = []
protalpha["qseq"] = []

charge_dict = {'D': -1, 'E': -1, 'R': +1, 'K': +1}  # This dict reproduces data from the paper

for prot in [h1, protalpha]:
    for i, d in enumerate(prot["seq"]):
        if charge_dict.get(d, None) is not None:
            prot["qseq"].append([charge_dict[d], i + 1])
    prot["qseq"] = np.array(prot["qseq"])

result = minimize(KdOpt, x0=kuhn, args=(expkD1[0], h1, protalpha, lb, kappa, 2, np.inf))

print(f"Minimization ended. OG b : {kuhn}, Min b : {result.x[0]}. Difference {kuhn - result.x[0]}")

iKd, integ, B2_O1, B2_O2 = findKd(h1, protalpha, lb, kappa, kuhn, order=2, ranget=np.inf)
fKd, integ, B2_O1, B2_O2 = findKd(h1, protalpha, lb, kappa, result.x[0], order=2, ranget=np.inf)

print("Initial Kd (um)", iKd*10**6, "Optimized Kd (um)", fKd*10**6)